In [ ]:
!pip install bs4 requests

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [ ]:
search_query = "laptop"

In [ ]:
url = f"https://www.amazon.co.jp/s?k={search_query}"

In [ ]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 " +
                  "(KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

In [ ]:
webpage=requests.get(url,headers=headers)

In [ ]:
webpage

<Response [200]>

In [ ]:
content=webpage.content
content

b'<!doctype html><html lang="en-us" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-fe.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n<!-- sp:end-feature:cs-optimization -->\n<!-- sp:feature:csm:head-open-part2 -->\n<script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function()

In [ ]:
soup = BeautifulSoup(webpage.content, "html.parser")
soup

<!DOCTYPE html>
<html class="a-no-js" data-19ax5a9jf="dingo" lang="en-us"><!-- sp:feature:head-start -->
<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>
<!-- sp:end-feature:head-start -->
<!-- sp:feature:csm:head-open-part1 -->
<script type="text/javascript">var ue_t0=ue_t0||+new Date();</script>
<!-- sp:end-feature:csm:head-open-part1 -->
<!-- sp:feature:cs-optimization -->
<meta content="on" http-equiv="x-dns-prefetch-control"/>
<link href="https://images-fe.ssl-images-amazon.com" rel="dns-prefetch"/>
<link href="https://m.media-amazon.com" rel="dns-prefetch"/>
<link href="https://completion.amazon.com" rel="dns-prefetch"/>
<!-- sp:end-feature:cs-optimization -->
<!-- sp:feature:csm:head-open-part2 -->
<script type="text/javascript">
window.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;
if (window.ue_ihb === 1) {

var ue_csm = window,
    ue_hob = +new Date();
(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=func

In [ ]:
links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

In [ ]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})

        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()

    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"

    return available

In [ ]:
links_list = []

In [ ]:
for link in links:
            links_list.append(link.get('href'))

d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

In [ ]:
for link in links:
      links_list.append(link.get('href'))

d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

# Loop for extracting product details from each link
for link in links_list:
  new_webpage = requests.get("https://www.amazon.co.jp" + link, headers=HEADERS)

  new_soup = BeautifulSoup(new_webpage.content, "html.parser")

  # Function calls to display all necessary product information
  d['title'].append(get_title(new_soup))
  d['price'].append(get_price(new_soup))
  d['rating'].append(get_rating(new_soup))
  d['reviews'].append(get_review_count(new_soup))
  d['availability'].append(get_availability(new_soup))


In [ ]:
amazon_df = pd.DataFrame.from_dict(d)
amazon_df['title'].replace('', np.nan, inplace=True)
amazon_df = amazon_df.dropna(subset=['title'])
amazon_df.to_csv("amazon_data.csv", header=True, index=False)

<ipython-input-79-967dc4f890b4>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  amazon_df['title'].replace('', np.nan, inplace=True)


In [ ]:
amazon_df[:20]

,title,price,rating,reviews,availability
0,TPV Office 2024 15.6 Inch 1920x1080 IPS LCD Wi...,,4.5 out of 5 stars,65 ratings,In Stock
1,"SERYUB 14"" Laptop, MS Office 2019, Windows 11 ...",,4.0 out of 5 stars,387 ratings,In Stock
2,"15.6 Inch N5095 Laptop, Crispy Operation, PC, ...",,4.3 out of 5 stars,148 ratings,Only 1 left in stock - order soon.
3,"ACEMAGIC Laptop N97 Laptop, 17.3 inch, Intel N...",,4.4 out of 5 stars,93 ratings,
4,SERYUB Core i7 11th Generation Core i7-1165G7 ...,,4.4 out of 5 stars,253 ratings,Only 17 left in stock - order soon.
5,Laptop (Windows 11 & Office 2019 Included) 15....,,4.2 out of 5 stars,183 ratings,Only 6 left in stock - order soon.
6,"TPV Laptop, 12th Generation, N100, Thin Laptop...",,4.6 out of 5 stars,14 ratings,Only 17 left in stock - order soon.
7,GM-JAPAN Laptop with Windows 11 Office 14.1 In...,,4.0 out of 5 stars,65 ratings,Only 7 left in stock (more on the way).
8,"Lenovo IdeaPad Slim 170 15.6"" AMD Ryzen™ 5 562...",,4.4 out of 5 stars,20 ratings,
9,SERYUB 15.6-inch Core i7 11th Generation Lapto...,,4.4 out of 5 stars,253 ratings,In Stock


In [ ]:
amazon_df.to_excel('amazon_products.xlsx', index=False)

print("✅ Excel file saved as 'amazon_products.xlsx'")

✅ Excel file saved as 'amazon_products.xlsx'
